In [17]:
import json
import pandas as pd
  
df = pd.read_json("/Users/oceane/Desktop/plan-your-trip-with-kayak/coordinates.json")

df[['latitute','longitude']] = df.coordinates.apply(lambda x: pd.Series(str(x).split(",")))
df = df.drop(columns=['coordinates', 'city'])


In [18]:
df = df.astype(float)

In [24]:
import requests
weather = []
for index, row in df.iterrows():
    appid = '7e208a67c297443752c742b0cdd73fdf'
    lat= row['latitute']
    lon= row['longitude']
    # URL = 'https://api.openweathermap.org/data/3.0/onecall'
    # PARAMS = {'lat':lat, 'lon':lon, 'appid': appid, 'units': 'metrics', 'cnt':7, 'exclude':'urrent,minutely,hourly,alerts'}
    # r = requests.get(url = URL, params=PARAMS).json()
    URL = f'https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&units=metric&exclude=current,minutely,hourly,alerts&appid={appid}'
    r = requests.get(URL).json()
    weather.append(r)
   


    

In [26]:

with open('weather.json', 'w') as f:
    json.dump(weather, f)


In [20]:
import requests

appid = '7e208a67c297443752c742b0cdd73fdf'
lat= '2.320041021720076'
lon='48.8588897'
URL = 'https://api.openweathermap.org/data/3.0/onecall'
PARAMS = {'lat':lat, 'lon':lon, 'appid': appid, 'units': 'metrics', 'cnt':7, 'exclude':'urrent,minutely,hourly,alerts'}
r = requests.get(url = URL, params=PARAMS).json()
#URL = f'https://api.openweathermap.org/data/3.0/onecall?lat=33.44&lon=-94.04&units=metric&exclude=current,minutely,hourly,alerts&appid={appid}'
#r = requests.get(URL).json()
r


{'lat': 2.32,
 'lon': 48.8589,
 'timezone': 'Etc/GMT-3',
 'timezone_offset': 10800,
 'current': {'dt': 1667169283,
  'sunrise': 1667183220,
  'sunset': 1667226553,
  'temp': 299.45,
  'feels_like': 299.45,
  'pressure': 1011,
  'humidity': 77,
  'dew_point': 295.1,
  'uvi': 0,
  'clouds': 13,
  'visibility': 10000,
  'wind_speed': 2.27,
  'wind_deg': 168,
  'wind_gust': 2.2,
  'weather': [{'id': 801,
    'main': 'Clouds',
    'description': 'few clouds',
    'icon': '02n'}]},
 'daily': [{'dt': 1667203200,
   'sunrise': 1667183220,
   'sunset': 1667226553,
   'moonrise': 1667203500,
   'moonset': 1667247900,
   'moon_phase': 0.22,
   'temp': {'day': 299.8,
    'min': 299.43,
    'max': 300.04,
    'night': 299.75,
    'eve': 300.04,
    'morn': 299.43},
   'feels_like': {'day': 299.8,
    'night': 299.75,
    'eve': 302.06,
    'morn': 299.43},
   'pressure': 1013,
   'humidity': 75,
   'dew_point': 295.09,
   'wind_speed': 4.34,
   'wind_deg': 163,
   'wind_gust': 4.01,
   'weather': [